In [1]:
import serial
import time

In [2]:
import math

In [3]:
def flush(ser):
    time.sleep(0.1)
    msg = ser.read_all()
    print(msg)

In [4]:
def testmode(ser):
    ser.write(b'testmode on\n')
    flush(ser)

In [5]:
class Location:
    def __init__(self, x=0, y=0, theta=0):
        self.x = x
        self.y = y
        self.theta =theta
    def add(self, x, y , theta):
        self.x += x
        self.y += y
        self.theta += theta
        self.theta = self.theta %(math.pi*2)
    def wheels(self, l,r):
        dtheta = (r-l)/235
        if dtheta == 0:
            dxx=((r+l)/2) * math.cos(self.theta)
            dyy=((r+l)/2) * math.sin(self.theta)
        else:
            raio=(r+l)/2/dtheta
            dx=raio * math.sin(dtheta)
            dy=raio * (1-math.cos(dtheta))
            dxx = dx*math.cos(self.theta) - dy*math.sin(self.theta)
            dyy = dx*math.sin(self.theta) + dy*math.cos(self.theta)
        self.add(dxx,dyy,dtheta)
    def __repr__(self):
        return '%.2f %.2f %.2f'%(self.x, self.y, self.theta/math.pi*180)

In [6]:
location=Location()
location

0.00 0.00 0.00

In [10]:
location.wheels(100+235*math.pi/2,100)
location

0.00 0.00 0.00

In [119]:
location

-181.16 1268.13 270.00

In [154]:
def setmotor(ser, l, r, speed):
    ser.write(b'setmotor lwheeldist %i rwheeldist %i speed %i\n'%(l, r, speed))
    location.wheels(l,r)
    time.sleep(max(abs(l),abs(r))/speed+0.1)
    flush(ser)
def forward(ser, dist, speed):
    setmotor(ser, dist, dist, speed)
def turn(angle, speed):
    num = angle/90*186/180*math.pi
    setmotor(ser, -num, num, speed)

In [155]:
def setldsrotation(ser):
    ser.write(b'setldsrotation on\n')
    flush(ser)

In [ ]:
def getldsscan(ser):
    flush(ser)
    ser.write(b'getldsscan\n')
    time.sleep(0.5)
    msg = ser.read_all()
    msgs = msg.split()
    return mapscan(msgs)

In [ ]:
import collections

In [172]:
Scan = collections.namedtuple('Scan', ['dist', 'intensity'])

In [ ]:
def mapscan(lines):
    result = dict()
    if len(lines)> 3:
        for line in lines:
            if line.startswith(b'getldsscan'):
                continue
            if line.startswith(b'Angle'):
                continue
            values = line.split(b',')
            if len(values) == 4:
                angle,dist,intensity,error = values
                if error == b'0':
                    result[angle]=Scan(int(dist),int(intensity))
    return result

In [165]:
sample=[b'getldsscan',
 b'AngleInDegrees,DistInMM,Intensity,ErrorCodeHEX',
 b'0,0,0,8035',
 b'1,1426,17,0',
 b'2,0,17,8021',
 b'3,4097,36,0',
 b'4,4093,41,0',
 b'5,4106,35,0',
 b'6,4087,28,0',
 b'7,4100,21,0',
 b'8,0,0,8035',
 b'9,2304,7,0',
 b'10,4140,9,0',
 b'11,0,0,8035',
 b'12,4171,13,0',
 b'13,4155,15,0',
 b'14,4146,9,0',
 b'15,0,0,8035',
 b'16,0,0,8035',
 b'17,4245,11,0',
 b'18,4252,12,0',
 b'19,4265,11,0',
 b'20,4289,11,0',
 b'21,0,0,8035',
 b'22,0,0,8035',
 b'23,0,0,8035',
 b'24,0,0,8035',
 b'25,0,0,8035',
 b'26,0,0,8021',
 b'27,4203,7,0',
 b'28,0,0,8035',
 b'29,0,0,8035',
 b'30,0,0,8035',
 b'31,0,0,8035',
 b'32,0,0,8035',
 b'33,0,0,8035',
 b'34,0,0,8035',
 b'35,0,0,8035',
 b'36,0,0,8035',
 b'37,0,0,8035',
 b'38,0,0,8035',
 b'39,0,0,8035',
 b'40,0,0,8035',
 b'41,0,0,8035',
 b'42,0,0,8035',
 b'43,0,0,8035',
 b'44,0,0,8035',
 b'45,0,0,8035',
 b'46,0,0,8035',
 b'47,2865,12,0',
 b'48,2832,14,0',
 b'49,0,0,8035',
 b'50,0,0,8035',
 b'51,0,0,8035',
 b'52,0,0,8035',
 b'53,0,0,8035',
 b'54,0,0,8035',
 b'55,0,0,8035',
 b'56,0,0,8035',
 b'57,0,0,8035',
 b'58,0,0,8035',
 b'59,0,0,8035',
 b'60,0,0,8035',
 b'61,0,0,8035',
 b'62,0,0,8035',
 b'63,0,0,8035',
 b'64,0,0,8035',
 b'65,0,0,8035',
 b'66,0,0,8035',
 b'67,0,0,8035',
 b'68,0,0,8035',
 b'69,0,0,8035',
 b'70,0,0,8035',
 b'71,4684,10,0',
 b'72,4609,10,0',
 b'73,0,0,8035',
 b'74,4582,10,0',
 b'75,4529,10,0',
 b'76,4477,7,0',
 b'77,0,0,8035',
 b'78,4481,9,0',
 b'79,4384,6,0',
 b'80,0,0,8035',
 b'81,0,0,8035',
 b'82,0,0,8035',
 b'83,0,0,8035',
 b'84,0,0,8035',
 b'85,0,0,8035',
 b'86,0,0,8035',
 b'87,4371,6,0',
 b'88,4416,12,0',
 b'89,4409,16,0',
 b'90,4339,20,0',
 b'91,0,0,8035',
 b'92,4360,17,0',
 b'93,4374,23,0',
 b'94,4402,21,0',
 b'95,4374,15,0',
 b'96,0,0,8035',
 b'97,4413,15,0',
 b'98,4399,15,0',
 b'99,4413,14,0',
 b'100,0,0,8035',
 b'101,0,0,8035',
 b'102,4406,14,0',
 b'103,4430,10,0',
 b'104,4448,14,0',
 b'105,0,0,8035',
 b'106,0,0,8035',
 b'107,0,0,8035',
 b'108,0,0,8035',
 b'109,0,0,8035',
 b'110,730,149,0',
 b'111,0,0,8035',
 b'112,0,0,8035',
 b'113,0,0,8035',
 b'114,0,0,8035',
 b'115,0,0,8035',
 b'116,0,0,8035',
 b'117,0,0,8035',
 b'118,0,0,8035',
 b'119,0,0,8035',
 b'120,0,0,8035',
 b'121,0,0,8035',
 b'122,0,0,8035',
 b'123,0,0,8035',
 b'124,5126,7,0',
 b'125,0,0,8035',
 b'126,0,0,8035',
 b'127,0,0,8035',
 b'128,0,0,8035',
 b'129,0,0,8035',
 b'130,0,0,8035',
 b'131,0,0,8035',
 b'132,0,0,8035',
 b'133,0,0,8035',
 b'134,0,0,8035',
 b'135,0,0,8035',
 b'136,0,0,8035',
 b'137,0,0,8035',
 b'138,0,0,8035',
 b'139,0,0,8035',
 b'140,4617,7,0',
 b'141,0,0,8035',
 b'142,4492,7,0',
 b'143,4459,8,0',
 b'144,0,0,8035',
 b'145,0,0,8035',
 b'146,0,0,8021',
 b'147,0,0,8035',
 b'148,0,0,8035',
 b'149,4152,11,0',
 b'150,4060,12,0',
 b'151,4024,10,0',
 b'152,4001,12,0',
 b'153,0,0,8035',
 b'154,0,0,8035',
 b'155,0,0,8035',
 b'156,0,0,8035',
 b'157,0,0,8035',
 b'158,1347,102,0',
 b'159,1324,156,0',
 b'160,1205,99,0',
 b'161,1162,192,0',
 b'162,1154,147,0',
 b'163,1149,84,0',
 b'164,1137,25,0',
 b'165,0,0,8035',
 b'166,0,0,8035',
 b'167,0,0,8035',
 b'168,0,0,8035',
 b'169,0,0,8035',
 b'170,0,0,8035',
 b'171,0,0,8035',
 b'172,0,0,8035',
 b'173,3476,13,0',
 b'174,3487,16,0',
 b'175,3426,11,0',
 b'176,0,0,8035',
 b'177,0,0,8035',
 b'178,3461,25,0',
 b'179,3442,23,0',
 b'180,3444,21,0',
 b'181,3450,21,0',
 b'182,0,0,8035',
 b'183,0,0,8035',
 b'184,1279,21,0',
 b'185,1221,88,0',
 b'186,1207,40,0',
 b'187,1191,48,0',
 b'188,1198,9,0',
 b'189,671,22,0',
 b'190,670,17,0',
 b'191,0,0,8035',
 b'192,670,17,0',
 b'193,0,0,8035',
 b'194,0,0,8035',
 b'195,0,0,8035',
 b'196,0,0,8035',
 b'197,0,0,8035',
 b'198,0,0,8035',
 b'199,0,0,8035',
 b'200,0,0,8035',
 b'201,0,0,8035',
 b'202,0,0,8035',
 b'203,0,0,8035',
 b'204,0,0,8035',
 b'205,0,0,8035',
 b'206,0,0,8035',
 b'207,0,0,8035',
 b'208,983,67,0',
 b'209,1701,82,0',
 b'210,1698,113,0',
 b'211,1702,74,0',
 b'212,1732,11,0',
 b'213,1733,24,0',
 b'214,1712,12,0',
 b'215,4084,9,0',
 b'216,0,0,8035',
 b'217,0,0,8035',
 b'218,0,0,8035',
 b'219,4299,9,0',
 b'220,0,0,8035',
 b'221,0,0,8035',
 b'222,944,78,0',
 b'223,907,123,0',
 b'224,901,120,0',
 b'225,899,107,0',
 b'226,900,114,0',
 b'227,902,152,0',
 b'228,904,120,0',
 b'229,906,106,0',
 b'230,1024,29,0',
 b'231,1027,27,0',
 b'232,0,0,8035',
 b'233,0,0,8035',
 b'234,0,0,8035',
 b'235,0,0,8035',
 b'236,0,0,8035',
 b'237,0,0,8035',
 b'238,0,0,8035',
 b'239,16977,29,0',
 b'240,16973,30,0',
 b'241,0,0,8035',
 b'242,0,0,8035',
 b'243,570,418,0',
 b'244,550,343,0',
 b'245,544,477,0',
 b'246,543,296,0',
 b'247,547,425,0',
 b'248,560,204,0',
 b'249,0,0,8035',
 b'250,0,0,8035',
 b'251,0,0,8035',
 b'252,0,0,8035',
 b'253,0,0,8035',
 b'254,0,0,8035',
 b'255,0,0,8035',
 b'256,0,0,8035',
 b'257,0,0,8035',
 b'258,0,0,8035',
 b'259,812,203,0',
 b'260,0,0,8035',
 b'261,0,0,8035',
 b'262,4463,12,0',
 b'263,4481,13,0',
 b'264,4445,15,0',
 b'265,4430,13,0',
 b'266,4445,16,0',
 b'267,0,0,8035',
 b'268,0,0,8035',
 b'269,4427,21,0',
 b'270,4406,22,0',
 b'271,4423,20,0',
 b'272,4456,28,0',
 b'273,4416,26,0',
 b'274,4384,24,0',
 b'275,4402,18,0',
 b'276,4430,18,0',
 b'277,4388,19,0',
 b'278,4406,17,0',
 b'279,4395,13,0',
 b'280,4305,10,0',
 b'281,0,0,8035',
 b'282,0,0,8035',
 b'283,0,0,8035',
 b'284,0,0,8035',
 b'285,0,0,8035',
 b'286,0,0,8035',
 b'287,0,0,8035',
 b'288,0,0,80']

In [173]:
mapscan(sample)

{b'1': Scan(dist=1426, intensity=17),
 b'3': Scan(dist=4097, intensity=36),
 b'4': Scan(dist=4093, intensity=41),
 b'5': Scan(dist=4106, intensity=35),
 b'6': Scan(dist=4087, intensity=28),
 b'7': Scan(dist=4100, intensity=21),
 b'9': Scan(dist=2304, intensity=7),
 b'10': Scan(dist=4140, intensity=9),
 b'12': Scan(dist=4171, intensity=13),
 b'13': Scan(dist=4155, intensity=15),
 b'14': Scan(dist=4146, intensity=9),
 b'17': Scan(dist=4245, intensity=11),
 b'18': Scan(dist=4252, intensity=12),
 b'19': Scan(dist=4265, intensity=11),
 b'20': Scan(dist=4289, intensity=11),
 b'27': Scan(dist=4203, intensity=7),
 b'47': Scan(dist=2865, intensity=12),
 b'48': Scan(dist=2832, intensity=14),
 b'71': Scan(dist=4684, intensity=10),
 b'72': Scan(dist=4609, intensity=10),
 b'74': Scan(dist=4582, intensity=10),
 b'75': Scan(dist=4529, intensity=10),
 b'76': Scan(dist=4477, intensity=7),
 b'78': Scan(dist=4481, intensity=9),
 b'79': Scan(dist=4384, intensity=6),
 b'87': Scan(dist=4371, intensity=6),
 

In [158]:
!sudo chmod 777 /dev/ttyACM1

In [159]:
ser = serial.Serial('/dev/ttyACM1', 115200, bytesize=8, parity='N', stopbits=1)

In [160]:
flush(ser)
testmode(ser)
testmode(ser)

b''
b"testmode on\r\nUnknown Cmd: '~'\r\n\x1a"
b'testmode on\r\n'


In [173]:
setlocation(0,0,0)
location

[0, 0, 0]

In [162]:
forward(ser,100,100)
location

b'\x1asetmotor lwheeldist 100 rwheeldist 100 speed 100\r\n\x1a'


[100.0, 0.0, 0.0]

In [163]:
turn(90, 100)
location

b'setmotor lwheeldist -186 rwheeldist 186 speed 100\r\n\x1a'


[100.0, 0.0, 90.69799991007075]

In [ ]:
forward(ser,100,100)
location

In [174]:
setmotor(ser,20,30, 100)
location

b'setmotor lwheeldist 20 rwheeldist 30 speed 100\r\n\x1a'


[25.0, 0.0, 2.4381182771524395]

In [175]:
setmotor(ser,20,30, 100)
location

b'setmotor lwheeldist 20 rwheeldist 30 speed 100\r\n\x1a'


[49.97736873899353, 1.0635087570624373, 4.876236554304879]

In [176]:
setmotor(ser,20,30, 100)
location

b'setmotor lwheeldist 20 rwheeldist 30 speed 100\r\n\x1a'


[74.88688466888566, 3.1886007876462124, 7.314354831457319]

In [ ]:
setmotor(ser,20,30, 100)
location

b'setmotor lwheeldist 20 rwheeldist 30 speed 100\r\n\x1a'


[81.08544285290259, 248.54645669596576, 105.3267095729854]

In [65]:
setlocation(-1,-2,370)
addlocation(1,2,-10)
location

[0, 0, 0]

In [78]:
setldsrotation(ser)

b'setldsrotation on\r\n\x1a'


In [91]:
getldsscan(ser)

b'5\r\n288,2178,9,0\r\n289,2117,43,0\r\n290,2074,44,0\r\n291,0,0,8035\r\n292,0,0,8035\r\n293,0,0,8035\r\n294,0,0,8035\r\n295,0,0,8035\r\n296,0,0,8035\r\n297,2177,19,0\r\n298,5032,11,0\r\n299,0,0,8035\r\n300,5022,10,0\r\n301,0,0,8035\r\n302,5045,9,0\r\n303,5193,7,0\r\n304,0,0,8035\r\n305,5263,6,0\r\n306,2424,8,0\r\n307,2394,20,0\r\n308,2379,22,0\r\n309,0,0,8035\r\n310,5683,6,0\r\n311,0,0,8035\r\n312,0,0,8035\r\n313,0,0,8035\r\n314,1157,106,0\r\n315,1141,146,0\r\n316,1035,11,0\r\n317,1021,232,0\r\n318,1022,230,0\r\n319,1010,212,0\r\n320,0,0,8035\r\n321,0,0,8035\r\n322,0,0,8035\r\n323,0,0,8035\r\n324,0,0,8035\r\n325,0,0,8035\r\n326,0,0,8035\r\n327,0,0,8035\r\n328,0,0,8035\r\n329,0,0,8035\r\n330,0,0,8035\r\n331,0,0,8035\r\n332,0,0,8035\r\n333,4525,6,0\r\n334,4448,7,0\r\n335,4292,6,0\r\n336,0,0,8035\r\n337,0,0,8035\r\n338,0,0,8035\r\n339,0,0,8035\r\n340,0,0,8035\r\n341,0,0,8035\r\n342,4371,10,0\r\n343,4329,11,0\r\n344,4326,13,0\r\n345,4289,9,0\r\n346,0,0,8035\r\n347,0,0,8035\r\n348,0,0,8035

[b'getldsscan',
 b'AngleInDegrees,DistInMM,Intensity,ErrorCodeHEX',
 b'0,0,0,8035',
 b'1,1426,17,0',
 b'2,0,17,8021',
 b'3,4097,36,0',
 b'4,4093,41,0',
 b'5,4106,35,0',
 b'6,4087,28,0',
 b'7,4100,21,0',
 b'8,0,0,8035',
 b'9,2304,7,0',
 b'10,4140,9,0',
 b'11,0,0,8035',
 b'12,4171,13,0',
 b'13,4155,15,0',
 b'14,4146,9,0',
 b'15,0,0,8035',
 b'16,0,0,8035',
 b'17,4245,11,0',
 b'18,4252,12,0',
 b'19,4265,11,0',
 b'20,4289,11,0',
 b'21,0,0,8035',
 b'22,0,0,8035',
 b'23,0,0,8035',
 b'24,0,0,8035',
 b'25,0,0,8035',
 b'26,0,0,8021',
 b'27,4203,7,0',
 b'28,0,0,8035',
 b'29,0,0,8035',
 b'30,0,0,8035',
 b'31,0,0,8035',
 b'32,0,0,8035',
 b'33,0,0,8035',
 b'34,0,0,8035',
 b'35,0,0,8035',
 b'36,0,0,8035',
 b'37,0,0,8035',
 b'38,0,0,8035',
 b'39,0,0,8035',
 b'40,0,0,8035',
 b'41,0,0,8035',
 b'42,0,0,8035',
 b'43,0,0,8035',
 b'44,0,0,8035',
 b'45,0,0,8035',
 b'46,0,0,8035',
 b'47,2865,12,0',
 b'48,2832,14,0',
 b'49,0,0,8035',
 b'50,0,0,8035',
 b'51,0,0,8035',
 b'52,0,0,8035',
 b'53,0,0,8035',
 b'54,0,0

In [89]:
s.split()

[b'getldsscan',
 b'AngleInDegrees,DistInMM,Intensity,ErrorCodeHEX',
 b'0,0,0,8035',
 b'1,1424,16,0',
 b'2,1423,7,0',
 b'3,4097,37,0',
 b'4,4103,37,0',
 b'5,4100,32,0',
 b'6,4100,28,0',
 b'7,4097,19,0',
 b'8,0,0,8035',
 b'9,2297,8,0',
 b'10,4149,8,0',
 b'11,0,0,8035',
 b'12,4159,13,0',
 b'13,4171,12,0',
 b'14,4168,9,0',
 b'15,0,0,8035',
 b'16,0,0,8035',
 b'17,4229,12,0',
 b'18,4269,12,0',
 b'19,4289,11,0',
 b'20,4309,12,0',
 b'21,0,0,8035',
 b'22,0,0,8035',
 b'23,0,0,8035',
 b'24,0,0,8035',
 b'25,0,0,8035',
 b'26,4213,8,0',
 b'27,0,0,8035',
 b'28,0,0,8035',
 b'29,0,0,8035',
 b'30,0,0,8035',
 b'31,0,0,8035',
 b'32,0,0,8035',
 b'33,0,0,8035',
 b'34,0,0,8035',
 b'35,0,0,8035',
 b'36,0,0,8035',
 b'37,0,0,8035',
 b'38,0,0,8035',
 b'39,0,0,8035',
 b'40,0,0,8035',
 b'41,0,0,8035',
 b'42,0,0,8035',
 b'43,0,0,8035',
 b'44,0,0,8035',
 b'45,0,0,8035',
 b'46,0,0,8035',
 b'47,2848,13,0',
 b'48,2834,16,0',
 b'49,0,0,8035',
 b'50,0,0,8021',
 b'51,0,0,8035',
 b'52,0,0,8035',
 b'53,0,0,8035',
 b'54,0,0,